## The Basics of Using Pandas 

This notebook demonstrates some of the advantages of working with [Pandas](https://pandas.pydata.org) DataFrames and Series. The goals are:

- Learn how to get started using `pandas`,
- Describe some of it's nomenclature 
- Load a data set with `pandas`
- Describe how to answer simple questions about a data set.
- Show some common `pandas` operations on data sets

Pandas is a very complicated and powerful framework, and the goal of this notebook is to expose the bare minimum to get you started working on your own data sets. Let's get to it!

If you are brand new to Jupyter notebooks, checkout this [short overview of the notebook user interface](https://nbviewer.jupyter.org/github/jupyter/notebook/blob/master/docs/source/examples/Notebook/Notebook%20Basics.ipynb#Overview-of-the-Notebook-UI) to get up to speed. 

## Importing Pandas

Traditionally `pandas`, `numpy` and other scientific libraries are imported with a short two letter name. The idea is to keep from importing `*` into your program's namespace while not having to continually type the (possibly long) package name. When looking at other people's code, you will often see the following:
```
   import numpy as np
   import pandas as pd
   import matplotlib.pyplot as plt
```

Write your code however it makes sense to you, but you will likely run into this when reading other people's code.

In [1]:
%pip install pandas # this is a notebook automagic directive that installs pandas if necessary
import pandas as pd

# Pretend you didn't see these.
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)


Note: you may need to restart the kernel to use updated packages.


## Demonstration Data

The data set we're going to inspect and manipulate with `pandas` is in comma seperated values (CSV) format, which is a very common file format. The data set is a contrived list of customer orders.

### Details of the Demonstration Data
This data set is constructed using [Faker](https://github.com/joke2k/faker), which allows us to create very large and repeatable data sets without having to permanently store them. The `csv_data` function returns a CSV formatted string of randomly generated customer order data. We wrap it in a `io.StringIO` object to make it appear more `file`-like and allows us to provide the data to `pandas.read_csv` which expects a file name or `file`-like object.

In [2]:
%pip install faker
import customers
import io

customer_data = io.StringIO(customers.csv_data())

Note: you may need to restart the kernel to use updated packages.


## Reading the Data

`pandas` has astounding support for various data formats. Here, we use the function `pandas.read_csv` to build a `pandas.DataFrame` initialized with the contents of the file. There are quite a few optional arguments to `pandas.read_csv`, but often you don't need to use them to get a useful DataFrame. As you learn more about your data set, you can refine these arguments to reduce the amount of "cleaning" you need to apply to it.

In [3]:
df = pd.read_csv(customer_data)

## Inspecting a DataFrame

A `pandas.DataFrame` has rows and columns which makes it look like a 2-dimensional array or Excel spreadsheet, however in practice the `DataFrame` is more like a list of columns than a matrix. Each of the columns in a DataFrame is a `pandas.Series` object which is a 1-dimensional array with axis labels.

In the output below, we have the column names across the top, the index on the far left (0, 9) and the data for each row.

In [4]:
df   # a bare DataFrame in a cell is equivalent to `display(df)`

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Date of Birth
0,90-6790,2020-03-06,HI77-BR4,35.06,10,2020-04-29,Ronald Baker,"7379 Brandi Fords\nPort Amandamouth, ND 59565",richardsmegan5walker.com,+1-181-269-5921x1723,1951-09-05
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaN,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2003-01-22
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaN,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2014-10-01
3,31-6105,2020-03-07,NG57-OK6,887.84,6,2020-06-01,Taylor Castaneda,"5629 Le Centers\nCopelandtown, KS 97552",catherineallenhawkins.com,001-399-790-6359x3895,1910-09-15
4,90-0956,2020-03-08,GP92-RI2,888.51,10,2020-03-28,Sean Mccarthy,"256 Cooper Overpass Apt. 316\nBlakehaven, FL 5...",twardgmail.com,(309)219-5240,1958-02-02
5,96-1537,2020-03-04,WR26-BR7,229.62,7,NaN,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1987-11-19
6,89-9680,2020-03-04,RJ01-NJ0,61.06,10,2020-06-18,Christopher Park,"67751 Jon Common\nEast Reneeburgh, NE 38218",dpacheco9yahoo.com,009-038-0324,1998-04-22
7,62-0581,2020-03-12,JG47-BF7,372.58,1,2020-05-19,John Hill,"9862 Cisneros Run Apt. 070\nTaraport, RI 11157",david052yahoo.com,576-422-6883,1996-11-18
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaN,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,1966-08-29
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaN,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,1985-07-10


## DataFrame Columns

The columns property of a `pandas.DataFrame` is a `list`-like object which contains the column names in left to right order. 

In [5]:
df.columns

Index(['Order Number', 'Order Date', 'Inventory Number', 'Unit Price', 'Units',
       'Ship Date', 'Name', 'Address', 'Email', 'Phone Number',
       'Date of Birth'],
      dtype='object')

The **columns** property is a readable and writable property, which allows you to reorganize the columns of the DataFrame in whatever order you wish. For instance, here we take a copy of our source DataFrame, `rdf`, reverse the columns and update `rdf.columns`.

In [6]:
rdf = df.copy()
rdf.columns = reversed(list(rdf.columns))
rdf # notice the data was not re-arranged.

,Date of Birth,Phone Number,Email,Address,Name,Ship Date,Units,Unit Price,Inventory Number,Order Date,Order Number
0,90-6790,2020-03-06,HI77-BR4,35.06,10,2020-04-29,Ronald Baker,"7379 Brandi Fords\nPort Amandamouth, ND 59565",richardsmegan5walker.com,+1-181-269-5921x1723,1951-09-05
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaN,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2003-01-22
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaN,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2014-10-01
3,31-6105,2020-03-07,NG57-OK6,887.84,6,2020-06-01,Taylor Castaneda,"5629 Le Centers\nCopelandtown, KS 97552",catherineallenhawkins.com,001-399-790-6359x3895,1910-09-15
4,90-0956,2020-03-08,GP92-RI2,888.51,10,2020-03-28,Sean Mccarthy,"256 Cooper Overpass Apt. 316\nBlakehaven, FL 5...",twardgmail.com,(309)219-5240,1958-02-02
5,96-1537,2020-03-04,WR26-BR7,229.62,7,NaN,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1987-11-19
6,89-9680,2020-03-04,RJ01-NJ0,61.06,10,2020-06-18,Christopher Park,"67751 Jon Common\nEast Reneeburgh, NE 38218",dpacheco9yahoo.com,009-038-0324,1998-04-22
7,62-0581,2020-03-12,JG47-BF7,372.58,1,2020-05-19,John Hill,"9862 Cisneros Run Apt. 070\nTaraport, RI 11157",david052yahoo.com,576-422-6883,1996-11-18
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaN,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,1966-08-29
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaN,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,1985-07-10


## DataFrame Index

The **index** property is the other important DataFrame property for locating and describing your data. The index can be quite complicated, but in the case of our customer order data set, it's a simple monotonically increasing integer from 0 to 9. Later we'll see some more interesting ways to set the index to explore our data set. 

In [7]:
df.index

RangeIndex(start=0, stop=10, step=1)

# Common DataFrame Functions - `Describe`

The `describe` function will apply some simple statistical functions to numerical columns found in the data set. In our case, the data has two columns which have numerical data: **Unit Price** and **Units**.

In [8]:
df.describe()

,Unit Price,Units
count,10.000000,10.000000
mean,456.831000,6.200000
std,365.743293,3.047768
min,21.670000,1.000000
25%,103.200000,4.250000
50%,421.360000,5.500000
75%,839.997500,9.250000
max,888.510000,10.000000


# Common DataFrame Functions - `Info`

The `info` function prints a concise summary of a DataFrame's component columns. 

In [9]:
df.info(verbose=True, memory_usage=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order Number      10 non-null     object 
 1   Order Date        10 non-null     object 
 2   Inventory Number  10 non-null     object 
 3   Unit Price        10 non-null     float64
 4   Units             10 non-null     int64  
 5   Ship Date         5 non-null      object 
 6   Name              10 non-null     object 
 7   Address           10 non-null     object 
 8   Email             10 non-null     object 
 9   Phone Number      10 non-null     object 
 10  Date of Birth     10 non-null     object 
dtypes: float64(1), int64(1), object(9)
memory usage: 1008.0+ bytes


# DataFrame Property - Shape

The **shape** property is a tuple whose members are the length of the DataFrame in rows and the width in columns. 

In [10]:
df.shape # our data has 10 rows and 11 columns

(10, 11)

# Common DataFrame Functions - `Head` & `Tail`


The `head` and `tail` functions return the first **N** or last **N** rows of a DataFrame. 


In [11]:
df.head(2)

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Date of Birth
0,90-6790,2020-03-06,HI77-BR4,35.06,10,2020-04-29,Ronald Baker,"7379 Brandi Fords\nPort Amandamouth, ND 59565",richardsmegan5walker.com,+1-181-269-5921x1723,1951-09-05
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaN,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2003-01-22


In [12]:
df.tail(2)

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Date of Birth
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaN,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,1966-08-29
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaN,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,1985-07-10


# Common DataFrame Functions - `Sample`

The `sample` function is a quick way to get a random sample of the rows in your DataFrame. It's helpful for spot checking values you've updated in a DataFrame without suffering from confirmation bias induced by only checking the `head` or `tail` of a very long DataFrame.



In [13]:
df.sample(5)

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Date of Birth
4,90-0956,2020-03-08,GP92-RI2,888.51,10,2020-03-28,Sean Mccarthy,"256 Cooper Overpass Apt. 316\nBlakehaven, FL 5...",twardgmail.com,(309)219-5240,1958-02-02
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaN,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2014-10-01
5,96-1537,2020-03-04,WR26-BR7,229.62,7,NaN,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1987-11-19
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaN,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,1966-08-29
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaN,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2003-01-22


# Viewing a Subset of a DataFrame's Columns

Often times our data sets are "long" with many rows and "wide" with many columns, and it'd be easier to comprehend if we could only view a subset of columns. The DataFrame index operator `[]` takes a column name or list of column names as an argument and returns a "view" of the DataFrame narrowed to the columns specified. The index operator can also take a boolean array as input which we'll explore further down.


In [14]:
df[['Order Number', 'Order Date', 'Ship Date']]

,Order Number,Order Date,Ship Date
0,90-6790,2020-03-06,2020-04-29
1,66-7342,2020-03-15,NaN
2,47-6588,2020-03-04,NaN
3,31-6105,2020-03-07,2020-06-01
4,90-0956,2020-03-08,2020-03-28
5,96-1537,2020-03-04,NaN
6,89-9680,2020-03-04,2020-06-18
7,62-0581,2020-03-12,2020-05-19
8,32-6389,2020-03-12,NaN
9,97-6482,2020-03-11,NaN


# Viewing a Subset of a DataFrame's Rows

Here's where things get weird. Using a DataFrame as iterator doesn't do what we expect it to do:

In [15]:
for row_maybe in df:
    print(type(row_maybe), row_maybe)

<class 'str'> Order Number
<class 'str'> Order Date
<class 'str'> Inventory Number
<class 'str'> Unit Price
<class 'str'> Units
<class 'str'> Ship Date
<class 'str'> Name
<class 'str'> Address
<class 'str'> Email
<class 'str'> Phone Number
<class 'str'> Date of Birth


# Viewing DataFrame Rows using `iloc`

The `iloc` property (not function) is an iterator that selects rows based on their integer row value. The property can be addressed using square brackets and accepts a variety of selectors: integers, slices, lists or tuples of integers, boolean arrays, and functions. Finally, the selectors can be a tuple of slices that address rows first, columns second!

In [16]:
df.iloc[1] # single rows are returned as pandas.Series

Order Number                                         66-7342
Order Date                                        2020-03-15
Inventory Number                                    NP82-IX1
Unit Price                                            722.75
Units                                                      4
Ship Date                                                NaN
Name                                           Amanda Prince
Address             2673 Gay Garden\nSouth Gabriel, VT 18295
Email                             amanda72garcia-dickson.net
Phone Number                              105-121-9772x76875
Date of Birth                                     2003-01-22
Name: 1, dtype: object

In [17]:
df.iloc[2:4] # multiple rows are returned as pandas.DataFrame's

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Date of Birth
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaN,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2014-10-01
3,31-6105,2020-03-07,NG57-OK6,887.84,6,2020-06-01,Taylor Castaneda,"5629 Le Centers\nCopelandtown, KS 97552",catherineallenhawkins.com,001-399-790-6359x3895,1910-09-15


In [18]:
df.iloc[:,[9,0,5,2]] # all rows, columns 9, 0, 5, and 2

,Phone Number,Order Number,Ship Date,Inventory Number
0,+1-181-269-5921x1723,90-6790,2020-04-29,HI77-BR4
1,105-121-9772x76875,66-7342,NaN,NP82-IX1
2,+1-867-834-9510x36966,47-6588,NaN,BK35-VD9
3,001-399-790-6359x3895,31-6105,2020-06-01,NG57-OK6
4,(309)219-5240,90-0956,2020-03-28,GP92-RI2
5,+1-205-572-0324x3336,96-1537,NaN,WR26-BR7
6,009-038-0324,89-9680,2020-06-18,RJ01-NJ0
7,576-422-6883,62-0581,2020-05-19,JG47-BF7
8,001-884-971-6167x3607,32-6389,NaN,BM63-SE1
9,318-745-5865,97-6482,NaN,ZY23-PM7


# Viewing DataFram Rows with `loc`
The `loc` property (not function) is an interator that selects rows and columns based on their label(s) or a boolean array. `loc` works just like `iloc` which used indices of rows and columns, just referenced by their labels. In this case, our DataFrame's indices are integers so those are the "labels" that `loc` expects. Later examples will show how to re-index a DataFrame and this method of access will make more sense.

In [19]:
df.loc[[1,3,5,7], ['Name','Address']]

,Name,Address
1,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295"
3,Taylor Castaneda,"5629 Le Centers\nCopelandtown, KS 97552"
5,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...
7,John Hill,"9862 Cisneros Run Apt. 070\nTaraport, RI 11157"


# Creating New DataFrame Columns

Sometimes adding new columns to a DataFrame can be helpful when working with data sets. The DataFrame will create a new column for us when we reference a non-existent column name. In this example, we also show how we can perform operations on the contents of a column versus having to iterate through each value and apply the operation in a "traditional" pythonic data structure like a `list` or `dict` or the explicit use of `map`. 


In [20]:
df['Total Invoice'] = df['Unit Price'] * df['Units']
df

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Date of Birth,Total Invoice
0,90-6790,2020-03-06,HI77-BR4,35.06,10,2020-04-29,Ronald Baker,"7379 Brandi Fords\nPort Amandamouth, ND 59565",richardsmegan5walker.com,+1-181-269-5921x1723,1951-09-05,350.60
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaN,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2003-01-22,2891.00
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaN,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2014-10-01,2350.70
3,31-6105,2020-03-07,NG57-OK6,887.84,6,2020-06-01,Taylor Castaneda,"5629 Le Centers\nCopelandtown, KS 97552",catherineallenhawkins.com,001-399-790-6359x3895,1910-09-15,5327.04
4,90-0956,2020-03-08,GP92-RI2,888.51,10,2020-03-28,Sean Mccarthy,"256 Cooper Overpass Apt. 316\nBlakehaven, FL 5...",twardgmail.com,(309)219-5240,1958-02-02,8885.10
5,96-1537,2020-03-04,WR26-BR7,229.62,7,NaN,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1987-11-19,1607.34
6,89-9680,2020-03-04,RJ01-NJ0,61.06,10,2020-06-18,Christopher Park,"67751 Jon Common\nEast Reneeburgh, NE 38218",dpacheco9yahoo.com,009-038-0324,1998-04-22,610.60
7,62-0581,2020-03-12,JG47-BF7,372.58,1,2020-05-19,John Hill,"9862 Cisneros Run Apt. 070\nTaraport, RI 11157",david052yahoo.com,576-422-6883,1996-11-18,372.58
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaN,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,1966-08-29,86.68
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaN,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,1985-07-10,4395.40


# Removing Columns from a DataFrame

Removing a column is accomplished with the function `pandas.DataFrame.drop` which has a huge number of optional arguments that can be intimidating. In our example, let's remove the column **Date of Birth** since HR has decided that we won't be sending our customers birthday cards any more.

In [21]:
df.drop(columns='Date of Birth')

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Total Invoice
0,90-6790,2020-03-06,HI77-BR4,35.06,10,2020-04-29,Ronald Baker,"7379 Brandi Fords\nPort Amandamouth, ND 59565",richardsmegan5walker.com,+1-181-269-5921x1723,350.60
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaN,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2891.00
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaN,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2350.70
3,31-6105,2020-03-07,NG57-OK6,887.84,6,2020-06-01,Taylor Castaneda,"5629 Le Centers\nCopelandtown, KS 97552",catherineallenhawkins.com,001-399-790-6359x3895,5327.04
4,90-0956,2020-03-08,GP92-RI2,888.51,10,2020-03-28,Sean Mccarthy,"256 Cooper Overpass Apt. 316\nBlakehaven, FL 5...",twardgmail.com,(309)219-5240,8885.10
5,96-1537,2020-03-04,WR26-BR7,229.62,7,NaN,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1607.34
6,89-9680,2020-03-04,RJ01-NJ0,61.06,10,2020-06-18,Christopher Park,"67751 Jon Common\nEast Reneeburgh, NE 38218",dpacheco9yahoo.com,009-038-0324,610.60
7,62-0581,2020-03-12,JG47-BF7,372.58,1,2020-05-19,John Hill,"9862 Cisneros Run Apt. 070\nTaraport, RI 11157",david052yahoo.com,576-422-6883,372.58
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaN,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,86.68
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaN,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,4395.40


# But Was The Column Removed?

The column **Date of Birth** is missing in the output above, but we're about to learn something interesting about the `pandas` way of handling data. Many, but not all, `pandas` operations return a new instance by default. If we go back and inspect our DataFrame, we'll be surprised to see that the **Date Of Birth** column _is still there!_

In [22]:
df

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Date of Birth,Total Invoice
0,90-6790,2020-03-06,HI77-BR4,35.06,10,2020-04-29,Ronald Baker,"7379 Brandi Fords\nPort Amandamouth, ND 59565",richardsmegan5walker.com,+1-181-269-5921x1723,1951-09-05,350.60
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaN,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2003-01-22,2891.00
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaN,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2014-10-01,2350.70
3,31-6105,2020-03-07,NG57-OK6,887.84,6,2020-06-01,Taylor Castaneda,"5629 Le Centers\nCopelandtown, KS 97552",catherineallenhawkins.com,001-399-790-6359x3895,1910-09-15,5327.04
4,90-0956,2020-03-08,GP92-RI2,888.51,10,2020-03-28,Sean Mccarthy,"256 Cooper Overpass Apt. 316\nBlakehaven, FL 5...",twardgmail.com,(309)219-5240,1958-02-02,8885.10
5,96-1537,2020-03-04,WR26-BR7,229.62,7,NaN,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1987-11-19,1607.34
6,89-9680,2020-03-04,RJ01-NJ0,61.06,10,2020-06-18,Christopher Park,"67751 Jon Common\nEast Reneeburgh, NE 38218",dpacheco9yahoo.com,009-038-0324,1998-04-22,610.60
7,62-0581,2020-03-12,JG47-BF7,372.58,1,2020-05-19,John Hill,"9862 Cisneros Run Apt. 070\nTaraport, RI 11157",david052yahoo.com,576-422-6883,1996-11-18,372.58
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaN,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,1966-08-29,86.68
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaN,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,1985-07-10,4395.40


# inplace=True

Many `pandas` functions take a boolean keyword option called 'inplace' whose default is False. If we wanted to make sure that the 'Date of Birth' column is dropped in the source DataFrame, we would do it this way:

In [23]:
df.drop(columns='Date of Birth', inplace=True)
df

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Total Invoice
0,90-6790,2020-03-06,HI77-BR4,35.06,10,2020-04-29,Ronald Baker,"7379 Brandi Fords\nPort Amandamouth, ND 59565",richardsmegan5walker.com,+1-181-269-5921x1723,350.60
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaN,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2891.00
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaN,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2350.70
3,31-6105,2020-03-07,NG57-OK6,887.84,6,2020-06-01,Taylor Castaneda,"5629 Le Centers\nCopelandtown, KS 97552",catherineallenhawkins.com,001-399-790-6359x3895,5327.04
4,90-0956,2020-03-08,GP92-RI2,888.51,10,2020-03-28,Sean Mccarthy,"256 Cooper Overpass Apt. 316\nBlakehaven, FL 5...",twardgmail.com,(309)219-5240,8885.10
5,96-1537,2020-03-04,WR26-BR7,229.62,7,NaN,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1607.34
6,89-9680,2020-03-04,RJ01-NJ0,61.06,10,2020-06-18,Christopher Park,"67751 Jon Common\nEast Reneeburgh, NE 38218",dpacheco9yahoo.com,009-038-0324,610.60
7,62-0581,2020-03-12,JG47-BF7,372.58,1,2020-05-19,John Hill,"9862 Cisneros Run Apt. 070\nTaraport, RI 11157",david052yahoo.com,576-422-6883,372.58
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaN,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,86.68
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaN,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,4395.40


# Data Cleaning - Handling Dates

Our data set has a number of columns that have content that looks like a date, however if you recall the output of the `info` function, those columns' data type is 'object' which generally means 'string'.

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order Number      10 non-null     object 
 1   Order Date        10 non-null     object 
 2   Inventory Number  10 non-null     object 
 3   Unit Price        10 non-null     float64
 4   Units             10 non-null     int64  
 5   Ship Date         5 non-null      object 
 6   Name              10 non-null     object 
 7   Address           10 non-null     object 
 8   Email             10 non-null     object 
 9   Phone Number      10 non-null     object 
 10  Total Invoice     10 non-null     float64
dtypes: float64(2), int64(1), object(8)
memory usage: 1008.0+ bytes


Since the **Order Date** column has no null values, let's work with it first. Converting that column to a more programmtically friendly datatype is pretty easy using `pandas.to_datetime`, which takes a Series as an argument.

In [25]:
pd.to_datetime(df['Order Date'])

0   2020-03-06
1   2020-03-15
2   2020-03-04
3   2020-03-07
4   2020-03-08
5   2020-03-04
6   2020-03-04
7   2020-03-12
8   2020-03-12
9   2020-03-11
Name: Order Date, dtype: datetime64[ns]

It turns out we can replace columns in a DataFrame as long as they are the same shape. In this case the shape we are replacing is (1, 10) or a single column. So to convert the **Order Date** column to _datetime64_ we would write:

In [26]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order Number      10 non-null     object        
 1   Order Date        10 non-null     datetime64[ns]
 2   Inventory Number  10 non-null     object        
 3   Unit Price        10 non-null     float64       
 4   Units             10 non-null     int64         
 5   Ship Date         5 non-null      object        
 6   Name              10 non-null     object        
 7   Address           10 non-null     object        
 8   Email             10 non-null     object        
 9   Phone Number      10 non-null     object        
 10  Total Invoice     10 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 1008.0+ bytes
None


,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Total Invoice
0,90-6790,2020-03-06,HI77-BR4,35.06,10,2020-04-29,Ronald Baker,"7379 Brandi Fords\nPort Amandamouth, ND 59565",richardsmegan5walker.com,+1-181-269-5921x1723,350.60
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaN,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2891.00
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaN,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2350.70
3,31-6105,2020-03-07,NG57-OK6,887.84,6,2020-06-01,Taylor Castaneda,"5629 Le Centers\nCopelandtown, KS 97552",catherineallenhawkins.com,001-399-790-6359x3895,5327.04
4,90-0956,2020-03-08,GP92-RI2,888.51,10,2020-03-28,Sean Mccarthy,"256 Cooper Overpass Apt. 316\nBlakehaven, FL 5...",twardgmail.com,(309)219-5240,8885.10
5,96-1537,2020-03-04,WR26-BR7,229.62,7,NaN,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1607.34
6,89-9680,2020-03-04,RJ01-NJ0,61.06,10,2020-06-18,Christopher Park,"67751 Jon Common\nEast Reneeburgh, NE 38218",dpacheco9yahoo.com,009-038-0324,610.60
7,62-0581,2020-03-12,JG47-BF7,372.58,1,2020-05-19,John Hill,"9862 Cisneros Run Apt. 070\nTaraport, RI 11157",david052yahoo.com,576-422-6883,372.58
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaN,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,86.68
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaN,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,4395.40


# Data Cleaning - What the Heck is `NaN`?

Sometimes data sets have missing values and they show up as `NaN` and `NaT` objects in our DataFrames. `NaN` is short for "Not A Number", while `NaT` is short for "Not A Time". Looking at the **Ship Date** column, you can see we have five `NaN` values lurking in there. From the documentation for `pandas.read_csv`, blank CSV fields are interpreted as `NaN` automatically, so that's where they came frome.

Let's first convert that column to `datetime64` like we did for **Order Date**.

In [27]:
df['Ship Date'] = pd.to_datetime(df['Ship Date'])
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order Number      10 non-null     object        
 1   Order Date        10 non-null     datetime64[ns]
 2   Inventory Number  10 non-null     object        
 3   Unit Price        10 non-null     float64       
 4   Units             10 non-null     int64         
 5   Ship Date         5 non-null      datetime64[ns]
 6   Name              10 non-null     object        
 7   Address           10 non-null     object        
 8   Email             10 non-null     object        
 9   Phone Number      10 non-null     object        
 10  Total Invoice     10 non-null     float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(6)
memory usage: 1008.0+ bytes
None


,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Total Invoice
0,90-6790,2020-03-06,HI77-BR4,35.06,10,2020-04-29,Ronald Baker,"7379 Brandi Fords\nPort Amandamouth, ND 59565",richardsmegan5walker.com,+1-181-269-5921x1723,350.60
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaT,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2891.00
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaT,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2350.70
3,31-6105,2020-03-07,NG57-OK6,887.84,6,2020-06-01,Taylor Castaneda,"5629 Le Centers\nCopelandtown, KS 97552",catherineallenhawkins.com,001-399-790-6359x3895,5327.04
4,90-0956,2020-03-08,GP92-RI2,888.51,10,2020-03-28,Sean Mccarthy,"256 Cooper Overpass Apt. 316\nBlakehaven, FL 5...",twardgmail.com,(309)219-5240,8885.10
5,96-1537,2020-03-04,WR26-BR7,229.62,7,NaT,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1607.34
6,89-9680,2020-03-04,RJ01-NJ0,61.06,10,2020-06-18,Christopher Park,"67751 Jon Common\nEast Reneeburgh, NE 38218",dpacheco9yahoo.com,009-038-0324,610.60
7,62-0581,2020-03-12,JG47-BF7,372.58,1,2020-05-19,John Hill,"9862 Cisneros Run Apt. 070\nTaraport, RI 11157",david052yahoo.com,576-422-6883,372.58
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaT,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,86.68
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaT,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,4395.40


Now we've converted **Ship Date** from object to datetime, but we've traded `NaN` for `NaT`. It's a step in the right direction. Since the shipping date is missing, we can infer that these rows are orders that have not shipped yet. We can use a variant of the `drop` function, `pandas.DataFrame.dropna`, to build a DataFrame that only contains records for orders that have valid shipping dates. The `axis` argument tells `dropna` how to drop values out of the DataFrame; `axis=0` means remove any row that has a `NaN` in any of the column values, whereas `axis=1` tells `dropna` to remove any columns which contain a `NaN`.

In [28]:
shipped_orders = df.dropna(axis=0) # drop rows which contain NaN or NaT
shipped_orders

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Total Invoice
0,90-6790,2020-03-06,HI77-BR4,35.06,10,2020-04-29,Ronald Baker,"7379 Brandi Fords\nPort Amandamouth, ND 59565",richardsmegan5walker.com,+1-181-269-5921x1723,350.60
3,31-6105,2020-03-07,NG57-OK6,887.84,6,2020-06-01,Taylor Castaneda,"5629 Le Centers\nCopelandtown, KS 97552",catherineallenhawkins.com,001-399-790-6359x3895,5327.04
4,90-0956,2020-03-08,GP92-RI2,888.51,10,2020-03-28,Sean Mccarthy,"256 Cooper Overpass Apt. 316\nBlakehaven, FL 5...",twardgmail.com,(309)219-5240,8885.10
6,89-9680,2020-03-04,RJ01-NJ0,61.06,10,2020-06-18,Christopher Park,"67751 Jon Common\nEast Reneeburgh, NE 38218",dpacheco9yahoo.com,009-038-0324,610.60
7,62-0581,2020-03-12,JG47-BF7,372.58,1,2020-05-19,John Hill,"9862 Cisneros Run Apt. 070\nTaraport, RI 11157",david052yahoo.com,576-422-6883,372.58


# Using Boolean Series to Select Data

Before we figure out the unshipped orders, we need a short digression. I mentioned early that the DataFrame indexing operator ,`[]`, is very versatile. As we've seen, it can take a list of column labels and returns a DataFrame consisting of the selected Columns. It can also take a list of booleans.


In [29]:
first_and_last = [True] + [False] * 8 + [True]

df[first_and_last]

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Total Invoice
0,90-6790,2020-03-06,HI77-BR4,35.06,10,2020-04-29,Ronald Baker,"7379 Brandi Fords\nPort Amandamouth, ND 59565",richardsmegan5walker.com,+1-181-269-5921x1723,350.6
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaT,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,4395.4


## Logical Operations on Series

As we saw when created the **Total Invoice** you can use columns in math operations that work on all the values of a column at once. Columns can also be used in logical operations too! For instance, if we'd like to know if there are any order's with **Unit Price** less than $100.00?

In [30]:
df['Unit Price'] < 100.0

0     True
1    False
2    False
3    False
4    False
5    False
6     True
7    False
8     True
9    False
Name: Unit Price, dtype: bool

The returned object here is a `pandas.Series` populated with `bool` values that the result of the comparison on every item in the **Unit Price** column. 

In [31]:
(df['Unit Price'] < 100.0).sum() # it's a Series!

3

## Building a DataFrame of Unshipped Orders

Since we have an idea about selecting rows using a boolean series, it seems pretty clear cut that we should use a logical query on the **Ship Date** column looking for `NaT`. 

In [32]:
df['Ship Date'] == pd.NaT 
# doesn't work as planned, NaT and NaN comparisons are not intuitives

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: Ship Date, dtype: bool

All the comparisons in the series returned `False` and that is the _magic_ of `NaN`. I will refer you to https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html for a comprehensive discussion about how to deal with `NaN` and `NaT`.

Instead of using logical operators, we can use the function `pandas.Series.isnull`.

In [33]:
df['Ship Date'].isnull()
# this looks better, the isnull() function encapsulates all the wierdness of NaN and NaT for us.

0    False
1     True
2     True
3    False
4    False
5     True
6    False
7    False
8     True
9     True
Name: Ship Date, dtype: bool

Now that we have a good boolean series which identifies the `NaT` values in the data, we can build an inverse DataFrame for orders still pending.

In [34]:
not_shipped = df['Ship Date'].isnull()

pending_orders = df[not_shipped]
pending_orders

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Total Invoice
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaT,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2891.00
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaT,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2350.70
5,96-1537,2020-03-04,WR26-BR7,229.62,7,NaT,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1607.34
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaT,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,86.68
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaT,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,4395.40


# So How Can We Use These Derived DataFrames?

Now we can ask questions like, how much money did we make from our shipped orders?

In [35]:
shipped_orders['Total Invoice'].sum()

15545.920000000002

In [36]:
shipped_orders['Total Invoice'].describe()

count       5.000000
mean     3109.184000
std      3860.757698
min       350.600000
25%       372.580000
50%       610.600000
75%      5327.040000
max      8885.100000
Name: Total Invoice, dtype: float64

In [37]:
pending_orders['Total Invoice'].sum()

11331.12

In [38]:
pending_orders['Total Invoice'].describe()

count       5.000000
mean     2266.224000
std      1590.236764
min        86.680000
25%      1607.340000
50%      2350.700000
75%      2891.000000
max      4395.400000
Name: Total Invoice, dtype: float64

# Imposing Order

If we're a business, we probably want to fulfill our orders quickly and usually in the order they were received (ignoring stock availability and other issues). We now have a list of pending orders but we don't know what order they were received in.  Yet.

In [39]:
pending_orders = pending_orders.sort_values(by='Order Date')
pending_orders

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Total Invoice
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaT,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2350.70
5,96-1537,2020-03-04,WR26-BR7,229.62,7,NaT,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1607.34
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaT,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,4395.40
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaT,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,86.68
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaT,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2891.00


In [40]:
pending_orders.head(1) # first pending order

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Total Invoice
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaT,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2350.7


In [41]:
pending_orders.tail(1) # last pending order

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Total Invoice
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaT,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2891.0


# A More Involved Sort

Ok, sure, we want to fulfill the orders in the order they were received. But sometimes we need to prioritize how orders are processed based on how much money they will bring in. The next cell is a little complicated, but it's just a re-iteration of all the steps we took to create the `pending_orders` DataFrame from scratch. The interesting part here is `sort_values` which we've specified two columns to sort by and whether to sort ascending or descending values.  Since we want to ship product to the biggest customer first, we order by 'Total Invoice' in descending order and a secondary sort by 'Order Date' in ascending order.

In [42]:
big_fish = df[df['Ship Date'].isnull()].sort_values(by=['Total Invoice', 'Order Date'], ascending=[False, True])
big_fish

,Order Number,Order Date,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Total Invoice
9,97-6482,2020-03-11,ZY23-PM7,879.08,5,NaT,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,4395.40
1,66-7342,2020-03-15,NP82-IX1,722.75,4,NaT,Amanda Prince,"2673 Gay Garden\nSouth Gabriel, VT 18295",amanda72garcia-dickson.net,105-121-9772x76875,2891.00
2,47-6588,2020-03-04,BK35-VD9,470.14,5,NaT,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2350.70
5,96-1537,2020-03-04,WR26-BR7,229.62,7,NaT,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1607.34
8,32-6389,2020-03-12,BM63-SE1,21.67,4,NaT,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,86.68


Looks like Mr. Martin is at the head of the list and Mr. Richardson will get his order last. It's a dog-eat-dog world in business.

# Looking at the Index
You'll notice in all our sorting examples the index numbers of rows remained constant even after the data was sorted, only their positions had changed. The index of the first row of `big_fish` is still 9 even after it's been sorted, it's original position in the data when it was first ingested from the CSV file. It turns out we can do something a little smarter with the index.

Now that we know some more things about our data, like it has dates in three columns, we can call `pandas.read_csv` with some optional arguments that will parse those dates for us automatically and set the index to the 'Order Date' column. Additionally, we'll sort the data by **Order Date**.

In [43]:
customer_data.seek(0) # rewind the customer data file to the beginning

0

In [44]:
index_column = 'Order Date'
date_columns = ['Order Date', 'Ship Date', 'Date of Birth']

df = pd.read_csv(customer_data, index_col=index_column, parse_dates=date_columns).sort_values(by=index_column)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10 entries, 2020-03-04 to 2020-03-15
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order Number      10 non-null     object        
 1   Inventory Number  10 non-null     object        
 2   Unit Price        10 non-null     float64       
 3   Units             10 non-null     int64         
 4   Ship Date         5 non-null      datetime64[ns]
 5   Name              10 non-null     object        
 6   Address           10 non-null     object        
 7   Email             10 non-null     object        
 8   Phone Number      10 non-null     object        
 9   Date of Birth     10 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(6)
memory usage: 880.0+ bytes


,Order Number,Inventory Number,Unit Price,Units,Ship Date,Name,Address,Email,Phone Number,Date of Birth
Order Date,,,,,,,,,,
2020-03-04,47-6588,BK35-VD9,470.14,5,NaT,Thomas Wang,USCGC Mitchell\nFPO AP 19018,kristinodom8vargas.org,+1-867-834-9510x36966,2014-10-01
2020-03-04,96-1537,WR26-BR7,229.62,7,NaT,Jay Fowler,158 Franklin Mountain Apt. 263\nNew Thomasmout...,ddean6ross.biz,+1-205-572-0324x3336,1987-11-19
2020-03-04,89-9680,RJ01-NJ0,61.06,10,2020-06-18,Christopher Park,"67751 Jon Common\nEast Reneeburgh, NE 38218",dpacheco9yahoo.com,009-038-0324,1998-04-22
2020-03-06,90-6790,HI77-BR4,35.06,10,2020-04-29,Ronald Baker,"7379 Brandi Fords\nPort Amandamouth, ND 59565",richardsmegan5walker.com,+1-181-269-5921x1723,1951-09-05
2020-03-07,31-6105,NG57-OK6,887.84,6,2020-06-01,Taylor Castaneda,"5629 Le Centers\nCopelandtown, KS 97552",catherineallenhawkins.com,001-399-790-6359x3895,1910-09-15
2020-03-08,90-0956,GP92-RI2,888.51,10,2020-03-28,Sean Mccarthy,"256 Cooper Overpass Apt. 316\nBlakehaven, FL 5...",twardgmail.com,(309)219-5240,1958-02-02
2020-03-11,97-6482,ZY23-PM7,879.08,5,NaT,James Martin,"0026 Parker Spring\nSouth Bradberg, ND 60017",welchmichael1gmail.com,318-745-5865,1985-07-10
2020-03-12,62-0581,JG47-BF7,372.58,1,2020-05-19,John Hill,"9862 Cisneros Run Apt. 070\nTaraport, RI 11157",david052yahoo.com,576-422-6883,1996-11-18
2020-03-12,32-6389,BM63-SE1,21.67,4,NaT,Matthew Richardson,"42076 Adam Ramp\nKimberlyhaven, OK 60700",vvancemartinez.net,001-884-971-6167x3607,1966-08-29


In [45]:
df.index

DatetimeIndex(['2020-03-04', '2020-03-04', '2020-03-04', '2020-03-06',
               '2020-03-07', '2020-03-08', '2020-03-11', '2020-03-12',
               '2020-03-12', '2020-03-15'],
              dtype='datetime64[ns]', name='Order Date', freq=None)

# Pandas is Great for Working with Timeseries Data

Now that our data has a `DatetimeIndex` index, we can ask some other questions. For instance, how many orders a day did we receive? How many orders a week did we ship?


In [46]:
df['Order Number'].resample('D').count()

Order Date
2020-03-04    3
2020-03-05    0
2020-03-06    1
2020-03-07    1
2020-03-08    1
2020-03-09    0
2020-03-10    0
2020-03-11    1
2020-03-12    2
2020-03-13    0
2020-03-14    0
2020-03-15    1
Freq: D, Name: Order Number, dtype: int64

In [47]:
df['Ship Date'].resample('1W').count()

Order Date
2020-03-08    4
2020-03-15    1
Freq: W-SUN, Name: Ship Date, dtype: int64

# Now What?

I hope I've demonstrated how easy it is to get started with `pandas`, despite it's complexity and richness.

To recap:

- A `pandas.DataFrame` is a list of `pandas.Series` objects with a shared index.
- Most `pandas` operations return copys of DataFrames unless you use `inplace=True`.
- You can use `pandas.Series` in mathematical and logical statements to act on the entire column.
- CSV is super easy to use as source of data for `pandas`.


`pandas` resources that you might find helpful:
- https://
- https://
- https://
- https://